In [1]:
# 1. Install necessary libraries
# We use 'quiet' to keep the output clean, remove it if you want to see installation details
!pip install pandas nltk textblob --quiet

# 2. Import modules
import pandas as pd
import random
import json
from datetime import datetime
import nltk
from textblob import TextBlob # Good for simple sentiment detection (keeping mood positive)

# 3. Download required NLTK data for tokenizing text
nltk.download('punkt', quiet=True)
nltk.download('brown', quiet=True) # Useful corpus for reference

# 4. Define a simple configuration for the bot
BOT_CONFIG = {
    "bot_name": "Remi", # Short for Reminiscence
    "user_name": "User", # We can change this dynamically later
    "session_start": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
}

print(f"✅ Environment Ready for {BOT_CONFIG['bot_name']}!")
print(f"Session started at: {BOT_CONFIG['session_start']}")

✅ Environment Ready for Remi!
Session started at: 2026-02-06 21:50:08


From the code above, 

pandas: We will use this to save conversation logs (very important for analysis later).

textblob: A simple library to detect sentiment. If a user gets agitated (negative sentiment), we can program the bot to change the subject.

BOT_CONFIG: A dictionary to store basic settings we can reference later.

We now want to move to Step 2: Defining the "Reminiscence Knowledge Base" This is the brain the bot will draw from.

In [3]:
# Define the topics and specific reminiscence prompts
reminiscence_topics = {
    "childhood": [
        "What was your favorite game to play outside when you were young?",
        "Do you remember the house you grew up in? What was your favorite room?",
        "What was your favorite home-cooked meal as a child?",
        "Did you have any pets growing up? What were their names?"
    ],
    "career": [
        "What was your very first job? Did you enjoy it?",
        "What is a professional accomplishment you are most proud of?",
        "Who was the best boss or mentor you ever had?",
        "How did your work change over the years?"
    ],
    "hobbies": [
        "What hobbies have you enjoyed throughout your life?",
        "Is there a particular song or type of music that always makes you smile?",
        "Did you enjoy traveling? Where was your favorite place to visit?",
        "What is a book or movie that had a big impact on you?"
    ],
    "family": [
        "How did you meet your spouse or partner?",
        "What is a favorite holiday tradition your family had?",
        "Tell me about a memorable family vacation.",
        "What values were most important to your parents?"
    ]
}

# Simple function to get a random question from a topic
def get_prompt(topic):
    if topic in reminiscence_topics:
        return random.choice(reminiscence_topics[topic])
    else:
        return "I'd love to hear more about that. Can you tell me more?"

print("✅ Knowledge Base Loaded with topics:", list(reminiscence_topics.keys()))

✅ Knowledge Base Loaded with topics: ['childhood', 'career', 'hobbies', 'family']


We are keeping the conversation focused on therapeutic areas and later we can add other information like music, school, e.t.c.

Defining The Logic Functions:

Next, we are going to build the "Brain" It is a function that takes what the user says, analyzes it and decides the memory topic to trigger next.

In [4]:
def analyze_sentiment(text):
    """
    Returns a score from -1 (Negative) to 1 (Positive).
    """
    blob = TextBlob(text)
    return blob.sentiment.polarity

def detect_topic(text):
    """
    Scans the user's text for keywords to see if they are talking about a specific topic.
    """
    text = text.lower()
    
    # Simple keyword dictionary
    keywords = {
        "family": ["mom", "dad", "mother", "father", "sister", "brother", "grandma", "grandpa", "wife", "husband", "son", "daughter"],
        "career": ["job", "work", "boss", "office", "career", "company", "retired", "profession"],
        "childhood": ["school", "young", "child", "kid", "play", "game", "toy", "grow up"],
        "hobbies": ["music", "song", "book", "read", "travel", "sport", "garden", "paint", "hobby"]
    }
    
    for topic, words in keywords.items():
        if any(word in text for word in words):
            return topic
    
    return None

def get_bot_response(user_input):
    # 1. Check mood
    mood = analyze_sentiment(user_input)
    
    # 2. Check topic
    detected_topic = detect_topic(user_input)
    
    # 3. Formulate response
    # If the user seems very distressed (negative sentiment), change to a lighter topic
    if mood < -0.5:
        return "It sounds like that is a heavy memory. Would you like to talk about your favorite music or hobbies instead?"
    
    # If we detected a topic, ask a question about THAT topic
    if detected_topic:
        return f"You mentioned something related to {detected_topic}. {get_prompt(detected_topic)}"
    
    # If no topic detected, pick a random one to keep the conversation flowing
    else:
        new_topic = random.choice(list(reminiscence_topics.keys()))
        return f"That is fascinating. switching gears a bit... {get_prompt(new_topic)}"

print("✅ Logic Functions Defined: 'analyze_sentiment', 'detect_topic', and 'get_bot_response'")

✅ Logic Functions Defined: 'analyze_sentiment', 'detect_topic', and 'get_bot_response'


The above is important because of context awareness, i.e if the user says I loved my mum, the bot will detect family and ask a family question next. It acts as a safety valve i.e < -0.5 check if the patient gets upset, the bot tries to pivot to a happier subject(music/hobbies)

In [ ]:
we are now going to stitch all those pieces together into a running loop. This will be the main chat loop.

In [ ]:
# 1. Start the conversation
print(f"--- {BOT_CONFIG['bot_name']} is online! (Type 'quit' to exit) ---")
print(f"{BOT_CONFIG['bot_name']}: Hello! I'm {BOT_CONFIG['bot_name']}. I'm here to listen to your stories. How are you feeling today?")

while True:
    # 2. Get input from the user
    user_input = input("You: ")
    
    # 3. Check if the user wants to stop
    if user_input.lower() in ["quit", "exit", "bye"]:
        print(f"{BOT_CONFIG['bot_name']}: It was lovely chatting with you. Goodbye!")
        break
    
    # 4. Get the bot's response using our logic function
    response = get_bot_response(user_input)
    
    # 5. Print the response
    print(f"{BOT_CONFIG['bot_name']}: {response}")

--- Remi is online! (Type 'quit' to exit) ---
Remi: Hello! I'm Remi. I'm here to listen to your stories. How are you feeling today?


You:  I am feeling great!


Remi: That is fascinating. switching gears a bit... Tell me about a memorable family vacation.


You:  I once went for a vacation to the moon with my parents in 1960


Remi: That is fascinating. switching gears a bit... What was your favorite home-cooked meal as a child?


...

Remi: It sounds like that is a heavy memory. Would you like to talk about your favorite music or hobbies instead?


You:  I don't want to


Remi: That is fascinating. switching gears a bit... How did your work change over the years?


You:  stop wasting my time


Remi: That is fascinating. switching gears a bit... Is there a particular song or type of music that always makes you smile?
